In [4]:
# coding=utf-8
# 导入环境
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import paddle
from paddle.io import Dataset
from paddle.nn import Conv2D, MaxPool2D, Linear, Dropout, BatchNorm, AdaptiveAvgPool2D, AvgPool2D
import paddle.nn.functional as F
import paddle.nn as nn

##########################################################################################
##########################################################################################
# 图像分块、Embedding
# 定义了一个名为 PatchEmbed 的自定义图像块嵌入层。它将输入图像分成小的块，
# 并将每个块通过一个卷积层进行投影（Embedding），从而将图像转换为一个序列形式的张量。
##########################################################################################
# PatchEmbed 类继承自 paddle.nn.Layer，并实现了 forward 方法来定义前向传播操作。主要步骤如下：
# __init__ 方法：该方法初始化 PatchEmbed 类，接收以下参数：
# img_size：输入图像的大小，默认为 224，可以是一个整数或一个元组 [H, W]。
# patch_size：图像块的大小，默认为 16，同样可以是一个整数或一个元组 [H, W]。
# in_chans：输入图像的通道数，默认为 3（RGB图像）。
# embed_dim：块嵌入向量的维度，默认为 768。
##########################################################################################
# forward 方法：在前向传播过程中，通过卷积操作将图像块嵌入到一个向量序列中。具体步骤如下：
# 首先，输入图像的维度为 [B, C, H, W]，其中 B 是批次大小，C 是通道数，H 和 W 是图像的高度和宽度。
# 然后，代码通过一个卷积层 self.proj 将图像块投影（Embedding）到一个新的向量空间。
# 这个投影操作与图像块的大小和嵌入向量的维度有关。
# 接下来，通过 flatten 方法将卷积输出展平为一个形状为 [B, C, H*W] 的张量，其中 H*W 表示图像中的总块数。
# 最后，通过 transpose 方法将维度重新排列为 [B, H*W, C]，这样每个块就成为序列中的一个元素，
# 其中 H*W 表示序列的长度，C 是块嵌入向量的维度。
##########################################################################################
##########################################################################################
class PatchEmbed(nn.Layer):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        # 原始大小为int，转为tuple，即：img_size原始输入224，变换后为[224,224]
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        # 图像块的个数
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2D(
            in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        # in_chans：输入图像的通道数，默认为 3（RGB图像）。
        # embed_dim：块嵌入向量的维度，默认为 768。
        # kernel_size：卷积核大小，默认为 patch_size。
        # stride：卷积步长，默认为 patch_size。

    def forward(self, x):
        B, C, H, W = x.shape # 将x.shape的四个维度分别赋值给这四个变量，x是张量。
        assert H == self.img_size[0] and W == self.img_size[1], \
            "Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})." # 用于检查输入图像的大小是否与模型期望的大小相匹配。
        # [B, C, H, W] -> [B, C, H*W] ->[B, H*W, C]
        x = self.proj(x).flatten(2).transpose((0, 2, 1))
        # self.proj在__init__中定义为一个2D卷积核。
        # flatten(2)表示将第2维度的数据展平，即：[B, C, H*W]
        # transpose((0, 2, 1))表示将第1维度和第2维度的数据交换，即：[B, H*W, C]，这样刚好是序列数据的表示方式。
        return x

In [5]:
##########################################################################################
##########################################################################################
# Multi-head Attention
# Attention类实现了一个多头注意力(Multi-head Attention)模块，通常在自注意力机制(Self-Attention)中使用,
# 用于计算注意力权重，然后对输入序列进行加权求和。多头注意力允许模型同时关注不同的子空间的特征，从而提高模型的表达能力。
##########################################################################################
# Attention 类的初始化方法 __init__：
# dim：输入向量的维度。
# num_heads：多头注意力的头数，用于将输入特征拆分成多个子空间。
# qkv_bias：是否在线性层（self.qkv）中使用偏置项。
# qk_scale：q 和 k 的缩放因子，默认为 None。若不提供，则使用 head_dim 的倒数（head_dim = dim // num_heads）的平方根。
# attn_drop：在计算注意力矩阵时的 dropout 比例。
# proj_drop：在输出时的 dropout 比例。
##########################################################################################
# forward 方法：在前向传播中，执行以下操作：
# 获取输入张量的形状，其中 N 代表批次大小，C 代表输入向量的维度。
# 通过线性变换 self.qkv 将输入张量 x 转换为三个部分：Query (q)、Key (k) 和 Value (v)。
# 将 q, k, v 分别重塑为 (batch_size * num_heads, N, head_dim) 的张量。
# 通过Transposed Dot-Product Attention计算注意力矩阵。首先将q与k进行点积（dot product），再进行缩放（scaled），然后通过Softmax函数进行归一化，得到注意力权重。
# 对注意力矩阵应用Dropout（self.attn_drop）进行正则化。
# 将注意力权重与v进行点积，然后重新变换形状，最后执行线性变换和输出向量的Dropout，并返回结果。
##########################################################################################
##########################################################################################
class Attention(nn.Layer):
    def __init__(self,
                 dim,
                 num_heads=8,
                 qkv_bias=False,
                 qk_scale=None,
                 attn_drop=0.,
                 proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim**-0.5
        # 计算 q,k,v 的转移矩阵
        self.qkv = nn.Linear(dim, dim * 3, bias_attr=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        # 最终的线性层
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        N, C = x.shape[1:]
        # 线性变换
        qkv = self.qkv(x).reshape((-1, N, 3, self.num_heads, C //
                                   self.num_heads)).transpose((2, 0, 3, 1, 4))
        # 分割 query key value
        q, k, v = qkv[0], qkv[1], qkv[2]
        # Scaled Dot-Product Attention
        # Matmul + Scale
        attn = (q.matmul(k.transpose((0, 1, 3, 2)))) * self.scale
        # SoftMax
        attn = nn.functional.softmax(attn, axis=-1)
        attn = self.attn_drop(attn)
        # Matmul
        x = (attn.matmul(v)).transpose((0, 2, 1, 3)).reshape((-1, N, C))
        # 线性变换
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [ ]:
##########################################################################################
##########################################################################################
# Mlp类多层感知机（MLP）模型
# MLP 由多个全连接层和非线性激活函数组成，用于处理非线性数据和进行特征提取。
##########################################################################################
# __init__ 方法：初始化 MLP 类，接收一些超参数：
# in_features：输入特征的维度。
# hidden_features：隐藏层的特征维度，默认为 None，若未指定则使用 in_features。
# out_features：输出特征的维度，默认为 None，若未指定则使用 in_features。
# act_layer：激活函数的类型，默认为 GELU（Gaussian Error Linear Unit）。
# drop：Dropout 的概率，默认为 0，用于正则化。
##########################################################################################
# forward 方法：在前向传播中，执行以下操作：
# x = self.fc1(x)：将输入张量 x 通过第一个全连接层 self.fc1 进行线性变换。这个层将输入特征映射到隐藏层的特征空间中。
# x = self.act(x)：对线性变换的结果应用激活函数，非线性激活函数（在这里是 GELU）用于引入非线性性，增加模型的表达能力。
# x = self.drop(x)：在隐藏层输出后，通过 Dropout 层进行正则化，随机丢弃一些隐藏层单元，以减少过拟合的可能性。
# x = self.fc2(x)：将经过激活和正则化后的隐藏层输出再次通过另一个全连接层 self.fc2 进行线性变换。这个层将隐藏层特征映射到最终的输出特征空间中。
# x = self.drop(x)：最后，在输出层之后再次应用 Dropout 层，对输出进行正则化。
##########################################################################################
##########################################################################################
class Mlp(nn.Layer):
    def __init__(self,
                 in_features,
                 hidden_features=None,
                 out_features=None,
                 act_layer=nn.GELU,
                 drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        # 输入层：线性变换
        x = self.fc1(x)
        # 应用激活函数
        x = self.act(x)
        # Dropout
        x = self.drop(x)
        # 输出层：线性变换
        x = self.fc2(x)
        # Dropout
        x = self.drop(x)
        return x

In [ ]:
##########################################################################################
##########################################################################################
# 使用了DropPath（Stochastic Depth）来代替传统的Dropout结构，DropPath可以理解为一种特殊的 Dropout。
# 其作用是在训练过程中随机丢弃子图层（randomly drop a subset of layers），而在预测时正常使用完整的 Graph。

def drop_path(x, drop_prob=0., training=False):
    if drop_prob == 0. or not training:
        return x
    keep_prob = paddle.to_tensor(1 - drop_prob)
    shape = (paddle.shape(x)[0], ) + (1, ) * (x.ndim - 1)
    random_tensor = keep_prob + paddle.rand(shape, dtype=x.dtype)
    random_tensor = paddle.floor(random_tensor)
    output = x.divide(keep_prob) * random_tensor
    return output

class DropPath(nn.Layer):
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)

In [ ]:
class Block(nn.Layer):
    def __init__(self,
                 dim,
                 num_heads,
                 mlp_ratio=4.,
                 qkv_bias=False,
                 qk_scale=None,
                 drop=0.,
                 attn_drop=0.,
                 drop_path=0.,
                 act_layer=nn.GELU,
                 norm_layer='nn.LayerNorm',
                 epsilon=1e-5):
        super().__init__()
        self.norm1 = eval(norm_layer)(dim, epsilon=epsilon)
        # Multi-head Self-attention
        self.attn = Attention(
            dim,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=drop)
        # DropPath
        self.drop_path = DropPath(drop_path) if drop_path > 0. else Identity()
        self.norm2 = eval(norm_layer)(dim, epsilon=epsilon)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim,
                       hidden_features=mlp_hidden_dim,
                       act_layer=act_layer,
                       drop=drop)

    def forward(self, x):
        # Multi-head Self-attention， Add， LayerNorm
        x = x + self.drop_path(self.attn(self.norm1(x)))
        # Feed Forward， Add， LayerNorm
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x